In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tqdm import tqdm
import random, sys, os

In [3]:
import sparse_linear_lib as sll

In [4]:
device = torch.device("cuda:0")

In [5]:
import pandas as pd
import time

## Pair Linear approximation

In [6]:
Ns = [16, 64, 256, 1024, 4096] #, 16384]
seeds = [147, 258, 369, 321, 654, 987, 741, 852, 963, 159, 357, 951, 753]

In [7]:
# Ns = [16384]
# seeds = [147]

In [8]:
mse = nn.MSELoss()

In [9]:
TRAIN_STEPS = 20000 #2000
def train_model(model, optimizer, X, A):
    for i in range(TRAIN_STEPS):
        out = model(X)
        loss = mse(out, A)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i%1000 == 0:
            print(f"The MSE loss is : {float(mse(out,A))}")
            
    with torch.no_grad():
        start = time.time()
        out = model(X)
        tt = time.time()-start
    return out, tt

In [10]:
def get_svd_output(A, n_comp):
    U, S, V = torch.svd(A)

    with torch.no_grad():
        _U = U[:, :n_comp]
        _S = S[:n_comp]
        _V = V[:, :n_comp]
        
        start = time.time()
        out = torch.mm(torch.mm(_U, torch.diag(_S)), _V.t())
        tt = time.time()-start
    
#     S[n_comp:] *= 0
#     out = torch.mm(torch.mm(U, torch.diag(S)), V.t())
    return out, tt

In [11]:
def save_stats(df, out, A, method, seed, nparam, tim, filename):
    diff = (out.data-A).abs()
    
    mean, std = float(diff.mean()), float(diff.std())
    err = float(mse(out, A))

    df = df.append({"method":method, "seed":seed, "mse":err,
                    "mean":mean, "std":std, "params":nparam, "time":tim}, 
                   ignore_index=True)
    df_ = df.copy()
    df.to_csv(f"./outputs/{file_name}.csv")
    
    print(f"Saving... file:{file_name} method:{method}")
    return df_

In [12]:
df = pd.DataFrame(columns=['mse', 'mean', 'std'])
df

,mse,mean,std


In [13]:
class Add_PairLinears(nn.Module):
    
    def __init__(self, input_dim, num_adds):
        super().__init__()
        self.pair_mixers = []
        self.perm_indices = []
        for i in range(num_adds):
            m = sll.PairLinear_MixerBlock(input_dim, input_dim)
            self.pair_mixers.append(m)
            if i > 0:
                rm = torch.randperm(input_dim)
                self.perm_indices.append(rm)
                
        self.pair_mixers = nn.ModuleList(self.pair_mixers)
        
    def forward(self, x):
        y = torch.zeros_like(x)
        for i, m in enumerate(self.pair_mixers):
            if i > 0:
                _x = x[:, self.perm_indices[i-1]]
            else:
                _x = x
                
            y += m(_x)
        return y

In [14]:
# model = Add_PairLinears(N, 4).to(device)

In [15]:
class Stack_PairLinears(nn.Module):
    
    def __init__(self, input_dim, num_adds):
        super().__init__()
        self.pair_mixers = []
        self.perm_indices = []
        for i in range(num_adds):
            m = sll.PairLinear_MixerBlock(input_dim, input_dim)
            self.pair_mixers.append(m)
            if i > 0:
                rm = torch.randperm(input_dim)
                self.perm_indices.append(rm)
                
        self.pair_mixers = nn.ModuleList(self.pair_mixers)
        
    def forward(self, x):
        for i, m in enumerate(self.pair_mixers):
            if i == 0:
                x = m(x)
            else:
                x = m(x[:, self.perm_indices[i-1]])
        return x

In [16]:
def log_base(a, base):
    return np.log(a) / np.log(base)

In [17]:
# df.append({"mse":1.0, "mean":2.0, "std":4.0}, ignore_index=True)

In [18]:
# import warnings
# warnings.filterwarnings(action='once') 

import warnings
warnings.filterwarnings('ignore')

'''
FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"method":method, "seed":seed, "mse":err,
'''

'\nFutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.\n  df = df.append({"method":method, "seed":seed, "mse":err,\n'

In [ ]:
for N in Ns:
    X = torch.eye(N).to(device)
    
    df = pd.DataFrame(columns=['method', 'seed', 'mse', 'mean', 'std', 'params', 'time'])
    file_name = f'record_err_{N}'
        
    for SEED in seeds:
        print()
        print(f"Experiment N={N} SEED={SEED}")
        
        torch.manual_seed(SEED)
        A = torch.rand(N, N).to(device)*2-1
        ### For each method compute the stats
        
        #####################################################
        ##### First 2x2 factorization
        model = sll.PairLinear_MixerBlock(N, N).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'pair', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        #####################################################
        ##### First log(N) factorization
#         _m = int(np.ceil(np.log2(N)))

        #### sqrt(N)/2 
        _m = int(np.ceil(np.sqrt(N)/2))
        model = sll.BlockLinear_MixerBlock(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'block-sqrt-half', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        
        #####################################################
        ##### Second sqrt(N) factorization
        _m = int(np.ceil(np.sqrt(N)))
        model = sll.BlockLinear_MixerBlock(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'block-sqrt', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        
        #####################################################
        _m = int(np.ceil(_n_params/(N*2)))
        out, tim = get_svd_output(A, _m)

        n_params = N*_m*2
        df = save_stats(df, out, A, 'lowR-same-param', SEED, n_params, tim, file_name)
        #####################################################
        
        #####################################################
        _m = int(np.ceil(_n_params/N))
        out, tim = get_svd_output(A, _m)

        n_params = N*_m
        df = save_stats(df, out, A, 'lowR-samex2', SEED, n_params, tim, file_name)
        #####################################################
        
        #####################################################
        _m = N // 2
        out, tim = get_svd_output(A, _m)

        n_params = N*_m*2
        df = save_stats(df, out, A, 'lowR-half', SEED, n_params, tim, file_name)
        #####################################################
        
        #####################################################
        torch.cuda.empty_cache()
        if N > 1024: continue

        ##### Pair Linear models parallel addition
        _m = int(np.ceil(np.log2(N)))
        model = Add_PairLinears(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'pair-Add', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer
        
        #####################################################
        
        ##### Pair Linear models sequential composition
        _m = int(np.ceil(np.log2(N)))
        model = Stack_PairLinears(N, _m).to(device)        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
        
        #### Train model
        out, tim = train_model(model, optimizer, X, A)
        _n_params = sum(p.numel() for p in model.parameters())
        df = save_stats(df, out, A, 'pair-Seq', SEED, _n_params, tim, file_name)
        #####################################################
        del model, optimizer


Experiment N=16 SEED=147
The MSE loss is : 0.4491165280342102
The MSE loss is : 0.20023737847805023
The MSE loss is : 0.1958433836698532
The MSE loss is : 0.19584333896636963
The MSE loss is : 0.19584333896636963
The MSE loss is : 0.19584333896636963
The MSE loss is : 0.19584333896636963
The MSE loss is : 0.19584333896636963
The MSE loss is : 0.19584333896636963
The MSE loss is : 0.19584336876869202
The MSE loss is : 0.19584336876869202
The MSE loss is : 0.19584335386753082
The MSE loss is : 0.1958433985710144
The MSE loss is : 0.19584336876869202
The MSE loss is : 0.19584335386753082
The MSE loss is : 0.19584335386753082
The MSE loss is : 0.19584336876869202
The MSE loss is : 0.19584345817565918
The MSE loss is : 0.19584333896636963
The MSE loss is : 0.19584335386753082
Saving... file:record_err_16 method:pair
The MSE loss is : 0.4491165280342102
The MSE loss is : 0.2941505014896393
The MSE loss is : 0.2922055125236511
The MSE loss is : 0.2913629412651062
The MSE loss is : 0.29136192

The MSE loss is : 0.0009753293124958873
The MSE loss is : 0.0009567507659085095
The MSE loss is : 0.0009172537829726934
The MSE loss is : 0.0008938864921219647
The MSE loss is : 0.0009341479162685573
The MSE loss is : 0.0008734357543289661
The MSE loss is : 0.0008408044232055545
Saving... file:record_err_16 method:pair-Seq

Experiment N=16 SEED=369
The MSE loss is : 0.3989996910095215
The MSE loss is : 0.1503232717514038
The MSE loss is : 0.14614027738571167
The MSE loss is : 0.14613927900791168
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.1461392641067505
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613927900791168
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613927900791168
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613929390907288
The MSE loss is : 0.14613927900791168
The MS

The MSE loss is : 0.004094417672604322
The MSE loss is : 0.003976179752498865
The MSE loss is : 0.003829577937722206
The MSE loss is : 0.0036368416622281075
The MSE loss is : 0.0035730283707380295
The MSE loss is : 0.003373573999851942
The MSE loss is : 0.0033073516096919775
The MSE loss is : 0.0032525742426514626
The MSE loss is : 0.0032036807388067245
The MSE loss is : 0.003156241960823536
The MSE loss is : 0.0031450071837753057
The MSE loss is : 0.0030307143460959196
The MSE loss is : 0.002950448775663972
The MSE loss is : 0.002867298200726509
The MSE loss is : 0.002805024851113558
Saving... file:record_err_16 method:pair-Seq

Experiment N=16 SEED=654
The MSE loss is : 0.38886141777038574
The MSE loss is : 0.1812228560447693
The MSE loss is : 0.17495106160640717
The MSE loss is : 0.17494326829910278
The MSE loss is : 0.17494329810142517
The MSE loss is : 0.17494326829910278
The MSE loss is : 0.17494326829910278
The MSE loss is : 0.17494328320026398
The MSE loss is : 0.17494328320026

The MSE loss is : 7.715663059570943e-07
The MSE loss is : 2.976347843741678e-07
Saving... file:record_err_16 method:pair-Add
The MSE loss is : 0.37310969829559326
The MSE loss is : 0.008875211700797081
The MSE loss is : 0.0027337484061717987
The MSE loss is : 0.0016565074911341071
The MSE loss is : 0.0007464430527761579
The MSE loss is : 0.00032948862644843757
The MSE loss is : 0.00021393978386186063
The MSE loss is : 0.0001385626383125782
The MSE loss is : 0.0001308194041484967
The MSE loss is : 0.00014170113718137145
The MSE loss is : 6.158198812045157e-05
The MSE loss is : 5.1572736992966384e-05
The MSE loss is : 9.520571620669216e-05
The MSE loss is : 9.024006430990994e-05
The MSE loss is : 3.377209941390902e-05
The MSE loss is : 2.9479975637514144e-05
The MSE loss is : 2.963114638987463e-05
The MSE loss is : 3.390208803466521e-05
The MSE loss is : 2.4143084374372847e-05
The MSE loss is : 2.505116572137922e-05
Saving... file:record_err_16 method:pair-Seq

Experiment N=16 SEED=741
T

The MSE loss is : 8.353540081884603e-11
The MSE loss is : 1.0218498047720459e-08
The MSE loss is : 3.0711794352100696e-07
The MSE loss is : 6.395468972186791e-07
The MSE loss is : 3.6084564669636165e-08
The MSE loss is : 2.0687730639679103e-08
The MSE loss is : 8.595994245297334e-07
The MSE loss is : 1.9105374349237536e-07
The MSE loss is : 9.468757156128049e-08
The MSE loss is : 1.4491222088963696e-07
Saving... file:record_err_16 method:pair-Add
The MSE loss is : 0.3898633122444153
The MSE loss is : 0.010697370395064354
The MSE loss is : 0.004760080948472023
The MSE loss is : 0.003046631347388029
The MSE loss is : 0.002364989835768938
The MSE loss is : 0.0017843737732619047
The MSE loss is : 0.001550428569316864
The MSE loss is : 0.001423489535227418
The MSE loss is : 0.0013357073767110705
The MSE loss is : 0.0013378255534917116
The MSE loss is : 0.0013657801318913698
The MSE loss is : 0.001234433613717556
The MSE loss is : 0.001282531302422285
The MSE loss is : 0.0012634628219529986


...

The MSE loss is : 0.30614805221557617
The MSE loss is : 0.30614805221557617
The MSE loss is : 0.30614805221557617
The MSE loss is : 0.30614811182022095
The MSE loss is : 0.30614805221557617
The MSE loss is : 0.30614805221557617
The MSE loss is : 0.3061481714248657
The MSE loss is : 0.30614808201789856
Saving... file:record_err_256 method:pair
The MSE loss is : 0.3379421830177307
The MSE loss is : 0.29575836658477783
The MSE loss is : 0.2955612540245056
The MSE loss is : 0.2955511212348938
The MSE loss is : 0.29555076360702515
The MSE loss is : 0.29554715752601624
The MSE loss is : 0.2955471873283386
The MSE loss is : 0.2955471873283386
The MSE loss is : 0.2955471873283386
The MSE loss is : 0.2955471873283386
The MSE loss is : 0.29554715752601624
The MSE loss is : 0.29554715752601624
The MSE loss is : 0.29554715752601624
The MSE loss is : 0.29554715752601624
The MSE loss is : 0.29554715752601624
The MSE loss is : 0.2955471873283386
The MSE loss is : 0.29554715752601624
The MSE loss is :

The MSE loss is : 0.3041708171367645
The MSE loss is : 0.30417078733444214
The MSE loss is : 0.30417078733444214
The MSE loss is : 0.30417078733444214
The MSE loss is : 0.30417078733444214
The MSE loss is : 0.3041708767414093
The MSE loss is : 0.3041708171367645
The MSE loss is : 0.30417078733444214
The MSE loss is : 0.3041708171367645
The MSE loss is : 0.30417078733444214
The MSE loss is : 0.30417078733444214
Saving... file:record_err_256 method:pair
The MSE loss is : 0.33734333515167236
The MSE loss is : 0.29586702585220337
The MSE loss is : 0.2957000732421875
The MSE loss is : 0.2956843376159668
The MSE loss is : 0.2956843376159668
The MSE loss is : 0.2956843078136444
The MSE loss is : 0.2956843078136444
The MSE loss is : 0.2956843078136444
The MSE loss is : 0.2956843376159668
The MSE loss is : 0.2956843376159668
The MSE loss is : 0.2956843376159668
The MSE loss is : 0.2956843078136444
The MSE loss is : 0.2956843376159668
The MSE loss is : 0.2956843376159668
The MSE loss is : 0.2956

The MSE loss is : 0.3229726552963257
The MSE loss is : 0.3229457139968872
The MSE loss is : 0.3229377269744873
The MSE loss is : 0.3229323923587799
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
The MSE loss is : 0.32293111085891724
Saving... file:record_err_1024 method:pair
The MSE loss is : 0.3339892029762268
The MSE loss is : 0.32015591859817505
The MSE loss is : 0.3201518654823303
The MSE loss is : 0.3201512098312378
The MSE loss is : 0.3201512098312378
The MSE loss is : 0.3201512098312378
The MSE loss is : 0.3201512098312378
The MSE loss is : 0.3201512098312378
The MSE loss is : 0.3201512098312378
The MSE loss is : 0.3201512098312378
The MSE loss is : 0.

The MSE loss is : 0.32386431097984314
The MSE loss is : 0.32367217540740967
The MSE loss is : 0.3235931098461151
The MSE loss is : 0.3235494792461395
The MSE loss is : 0.3235347867012024
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235335946083069
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235335946083069
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
The MSE loss is : 0.3235336244106293
Saving... file:record_err_1024 method:pair
The MSE loss is : 0.33462899923324585
The MSE loss is : 0.3207998275756836
The MSE loss is : 0.3207966089248657
The MSE loss is : 0.3207966089248657
The MSE loss is : 0.3207966089248657
The MSE loss is : 0.3207966089248657
The MSE loss is : 0.3207966089248657
The MSE loss is : 0.320796608

The MSE loss is : 0.3239274322986603
The MSE loss is : 0.32345110177993774
The MSE loss is : 0.32324424386024475
The MSE loss is : 0.3231106698513031
The MSE loss is : 0.32305413484573364
The MSE loss is : 0.32303905487060547
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
The MSE loss is : 0.32303857803344727
Saving... file:record_err_1024 method:pair
The MSE loss is : 0.33410510420799255
The MSE loss is : 0.3202698230743408
The MSE loss is : 0.3202671408653259
The MSE loss is : 0.3202671408653259
The MSE loss is : 0.3202671408653259
The MSE loss is : 0.3202671408653259
The MSE loss i

In [ ]:
torch.cuda.empty_cache()

In [ ]:
int(np.ceil(np.log2(64)))

In [ ]:
np.log2(256)

In [ ]:
asfsdfasdf

In [ ]:
sum(p.numel() for p in model.parameters())